# Sarter Notebook

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

## Reading Data

In [32]:
# read train data
df_orders = pd.read_csv("orders.csv", sep=";")
df_products = pd.read_csv("product_attributes.csv", sep=",")
df_dists = pd.read_csv("cities_data.csv", sep=";")
df_ordersXproducts = pd.read_csv('ordersXproducts.csv')
del df_ordersXproducts['Unnamed: 0']

# read test data
df_test = pd.read_csv("test.csv", sep=";")

In [3]:
df_dists

,city_from_name,city_to_name,city_from_coord,city_to_coord,distance
0,Dusseldorf,Zaragoza,"(51.2254018, 6.7763137)","(41.6521342, -0.8809428)",1214.4772
1,Dusseldorf,Venlo,"(51.2254018, 6.7763137)","(51.39244885, 6.1511724144122955)",47.3876
2,Dusseldorf,Bratislava,"(51.2254018, 6.7763137)","(48.1516988, 17.1093063)",819.2894
3,Dusseldorf,Hamburg,"(51.2254018, 6.7763137)","(53.550341, 10.000654)",339.2215
4,Dusseldorf,Rome,"(51.2254018, 6.7763137)","(41.8933203, 12.4829321)",1125.0290
...,...,...,...,...,...
661,Rotterdam,Helsinki,"(51.9244424, 4.47775)","(60.1674881, 24.9427473)",1560.6484
662,Rotterdam,Prague,"(51.9244424, 4.47775)","(50.0874654, 14.4212535)",726.5575
663,Rotterdam,Valencia,"(51.9244424, 4.47775)","(39.4697065, -0.3763353)",1434.1499
664,Rotterdam,Bremen,"(51.9244424, 4.47775)","(53.0758196, 8.8071646)",320.6459


## Merge tables

In [33]:
df_orders.join(df_products.set_index('product_id'), on='product_id').to_csv('Datasets/ordersXproducts.csv')

In [34]:
df = pd.merge(df_ordersXproducts, df_dists, how='left', left_on=['origin_port', 'logistic_hub'], right_on=['city_from_name', 'city_to_name'])

In [35]:
del df['city_from_name']
del df['city_to_name']
df = df.rename(columns={'city_from_coord': 'origin_port_coord', 'city_to_coord': 'logistic_hub_coord', 'distance': 'dist_origin_hub'})

In [36]:
df = pd.merge(df, df_dists, how='left', left_on=['logistic_hub', 'customer'], right_on=['city_from_name', 'city_to_name'])

In [37]:
del df['city_from_name']
del df['city_to_name']
del df['city_from_coord']
df = df.rename(columns={'city_to_coord': 'customer_coord', 'distance': 'dist_hub_customer'})

In [38]:
df = pd.merge(df, df_dists, how='left', left_on=['origin_port', 'customer'], right_on=['city_from_name', 'city_to_name'])

In [39]:
del df['city_from_name']
del df['city_to_name']
del df['city_from_coord']
del df['city_to_coord']
df = df.rename(columns={'distance': 'dist_origin_customer'})

In [40]:
pd.isna(df.at[2, 'dist_origin_hub'])

True

In [41]:
df_dists[(df_dists['city_from_name'] == 'athens')&(df_dists['city_to_name'] == 'Venlo')]

,city_from_name,city_to_name,city_from_coord,city_to_coord,distance


In [42]:
for i in range(len(df)):
    if (not pd.isna(df.at[i, 'dist_origin_hub'])) and (not pd.isna(df.at[i, 'dist_hub_customer'])):
        df.at[i, 'dist_origin_customer'] = df.at[i, 'dist_origin_hub'] + df.at[i, 'dist_hub_customer']

In [14]:
df.head()

,order_id,origin_port,3pl,customs_procedures,logistic_hub,customer,product_id,units,late_order,weight,material_handling,origin_port_coord,logistic_hub_coord,dist_origin_hub,customer_coord,dist_hub_customer,dist_origin_customer
0,366c7a3d298f,Rotterdam,v_002,DTP,Venlo,Marseille,1692723,583,True,1778.0,5.0,"(51.9244424, 4.47775)","(51.39244885, 6.1511724144122955)",130.0459,"(43.2961743, 5.3699525)",902.0420,1032.0879
1,45f906331e10,Rotterdam,v_004,CRF,Rome,Marseille,1644308,459,False,1088.0,3.0,"(51.9244424, 4.47775)","(41.8933203, 12.4829321)",1269.2365,"(43.2961743, 5.3699525)",604.0216,1873.2581
2,ac84a6e4af0f,Athens,v_002,CRF,Venlo,Paris,1684170,464,True,505.0,4.0,NaN,NaN,NaN,"(48.8588897, 2.3200410217200766)",392.9249,NaN
3,f5e98cb29790,Rotterdam,v_004,CRF,Lille,Milan,1620510,678,False,1308.0,4.0,"(51.9244424, 4.47775)","(50.6365654, 3.0635282)",173.9644,"(45.4641943, 9.1896346)",733.8784,907.8428
4,a9e7c9bee35b,Barcelona,v_002,CRF,Venlo,Berlin,1699372,353,False,1465.0,0.0,NaN,NaN,NaN,"(52.5170365, 13.3888599)",512.8188,NaN


# PLOTS

In [44]:
from scipy.stats import pointbiserialr
#ax = plt.axes(projection='3d')
#ax.scatter3D(df['weight'], df['units'], df['late_order'])
df['tweight'] = df['weight']*df['units']
df.corr()


/tmp/ipykernel_46953/3981876502.py:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,product_id,units,late_order,weight,material_handling,dist_origin_hub,dist_hub_customer,dist_origin_customer,tweight
product_id,1.000000,-0.002913,-0.004933,0.058002,0.121832,0.002987,0.003430,0.000822,0.051624
units,-0.002913,1.000000,0.167897,-0.008776,-0.001853,-0.001118,-0.001315,-0.002678,0.399631
late_order,-0.004933,0.167897,1.000000,0.072996,-0.018592,0.099854,0.082055,0.109535,0.151933
weight,0.058002,-0.008776,0.072996,1.000000,0.016001,0.001999,0.003726,0.004841,0.905635
material_handling,0.121832,-0.001853,-0.018592,0.016001,1.000000,-0.000417,0.001832,0.004356,0.013871
dist_origin_hub,0.002987,-0.001118,0.099854,0.001999,-0.000417,1.000000,0.222371,0.738523,0.001549
dist_hub_customer,0.003430,-0.001315,0.082055,0.003726,0.001832,0.222371,1.000000,0.711474,0.002820
dist_origin_customer,0.000822,-0.002678,0.109535,0.004841,0.004356,0.738523,0.711474,1.000000,0.003447
tweight,0.051624,0.399631,0.151933,0.905635,0.013871,0.001549,0.002820,0.003447,1.000000
